In [1]:
import pickle
with open('./data_traffic/train.pickle', 'rb') as ftrain:
    datatrain= pickle.load(ftrain)

    
datatrain['features'].shape

(34799, 32, 32, 3)

In [2]:
pwd

'd:\\tensorflow'

In [3]:
import os
with open('./data_traffic/test.pickle', 'rb') as ftest:
    datatest = pickle.load(ftest)

with open('./data_traffic/valid.pickle', 'rb') as fval:
    dataval = pickle.load(fval)




In [11]:
xtrain = datatrain['features']
ytrain = datatrain['labels']

xtest = datatest['features']
ytest = datatest['labels']

xval = dataval['features']
yval = dataval['labels']

In [46]:
import matplotlib.pyplot as plt
plt.imshow(xtrain[6])
plt.show()

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [47]:
ytrain

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

xử lý để lấy nhãn

In [15]:
import pandas as pd
df = pd.read_csv('./data_traffic/label_names.csv')
id = df['ClassId']
names = df['SignName']
ClassName = {}
for key, value in zip(id, names):
    ClassName[key] = value
ClassName


{0: 'Speed limit (20km/h)',
 1: 'Speed limit (30km/h)',
 2: 'Speed limit (50km/h)',
 3: 'Speed limit (60km/h)',
 4: 'Speed limit (70km/h)',
 5: 'Speed limit (80km/h)',
 6: 'End of speed limit (80km/h)',
 7: 'Speed limit (100km/h)',
 8: 'Speed limit (120km/h)',
 9: 'No passing',
 10: 'No passing for vehicles over 3.5 metric tons',
 11: 'Right-of-way at the next intersection',
 12: 'Priority road',
 13: 'Yield',
 14: 'Stop',
 15: 'No vehicles',
 16: 'Vehicles over 3.5 metric tons prohibited',
 17: 'No entry',
 18: 'General caution',
 19: 'Dangerous curve to the left',
 20: 'Dangerous curve to the right',
 21: 'Double curve',
 22: 'Bumpy road',
 23: 'Slippery road',
 24: 'Road narrows on the right',
 25: 'Road work',
 26: 'Traffic signals',
 27: 'Pedestrians',
 28: 'Children crossing',
 29: 'Bicycles crossing',
 30: 'Beware of ice/snow',
 31: 'Wild animals crossing',
 32: 'End of all speed and passing limits',
 33: 'Turn right ahead',
 34: 'Turn left ahead',
 35: 'Ahead only',
 36: 'Go st

trộn ảnh và nhãn

In [16]:
ClassName[ytrain[0]]

'End of no passing'

In [18]:
from sklearn.utils import shuffle
xtrain, ytrain = shuffle(xtrain, ytrain)

In [19]:
ClassName[ytrain[0]]

'Keep right'

chuẩn hoá ảnh

In [20]:
xtrain = xtrain.astype("float") /255.0
xtest = xtest.astype("float") /255.0
xval = xval.astype('float')/255.0

chuẩn hoá nhãn

In [21]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
ytrain = lb.fit_transform(ytrain)
ytest = lb.transform(ytest)
yval = lb.fit_transform(yval)


In [22]:
ytrain[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

XÂY DỤNG MẠNG

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD


In [24]:

width, height,channel = xtrain[0].shape
num_class = len(ClassName)
img_shape = xtrain[0].shape

In [25]:
# model

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape = img_shape))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3), padding='same' ))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same' ))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(num_class))
model.add(Activation("softmax"))


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

xoay ảnh ngẫu nhiên

In [26]:
aug = ImageDataGenerator(rotation_range=0.18, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
learning_rate = 0.001
epochs = 15
batch_size = 64
optim = SGD(learning_rate=learning_rate, momentum=0.9)


In [27]:
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics="accuracy")

In [28]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3902132261648106820
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1749391771
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4288974331560625767
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [29]:
model.fit_generator(aug.flow(xtrain, ytrain, batch_size=batch_size), validation_data=(xval, yval), epochs=epochs, verbose=1 )

Epoch 1/15


C:\Users\hoang\AppData\Local\Temp\ipykernel_2180\3814667668.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(aug.flow(xtrain, ytrain, batch_size=batch_size), validation_data=(xval, yval), epochs=epochs, verbose=1 )


544/544 [==============================] - 20s 25ms/step - loss: 2.4155 - accuracy: 0.3409 - val_loss: 2.3037 - val_accuracy: 0.3299
Epoch 2/15
544/544 [==============================] - 14s 25ms/step - loss: 1.4438 - accuracy: 0.5608 - val_loss: 2.0123 - val_accuracy: 0.4483
Epoch 3/15
544/544 [==============================] - 14s 25ms/step - loss: 1.0717 - accuracy: 0.6621 - val_loss: 1.9734 - val_accuracy: 0.5005
Epoch 4/15
544/544 [==============================] - 18s 34ms/step - loss: 0.8371 - accuracy: 0.7394 - val_loss: 1.3884 - val_accuracy: 0.5839
Epoch 5/15
544/544 [==============================] - 15s 28ms/step - loss: 0.6790 - accuracy: 0.7882 - val_loss: 1.4187 - val_accuracy: 0.5855
Epoch 6/15
544/544 [==============================] - 14s 26ms/step - loss: 0.5467 - accuracy: 0.8305 - val_loss: 1.1384 - val_accuracy: 0.6562
Epoch 7/15
544/544 [==============================] - 17s 31ms/step - loss: 0.4644 - accuracy: 0.8576 - val_loss: 1.0788 - val_accuracy: 0.6821
Epo

In [30]:
model.save('traffic.h5')

In [33]:
model.evaluate(xtest, ytest, batch_size=64)

198/198 [==============================] - 1s 5ms/step - loss: 0.5505 - accuracy: 0.8383


[0.5505223870277405, 0.8383214473724365]

In [41]:
y_pre = model.predict(xtest[1:2])
y_pre

1/1 [==============================] - 0s 19ms/step


array([[3.64840176e-04, 9.99455750e-01, 5.32261729e-05, 9.39232962e-07,
        1.05915326e-07, 9.06902278e-06, 7.19176478e-06, 5.54975145e-07,
        4.40532602e-07, 2.59868616e-09, 1.30038691e-08, 1.39265282e-07,
        3.55643351e-07, 7.20950197e-08, 8.56870156e-06, 3.26243246e-08,
        6.27134796e-07, 4.52990349e-07, 8.51985078e-06, 1.05852521e-07,
        6.05888602e-08, 1.08254437e-06, 7.50420259e-09, 1.54211307e-07,
        3.76312273e-05, 2.57218471e-06, 7.34506713e-08, 2.17734782e-06,
        1.30077333e-05, 2.97518341e-07, 2.13578716e-07, 2.47836098e-08,
        1.89937836e-07, 1.19221639e-07, 4.24239460e-07, 6.17941259e-06,
        7.70250267e-07, 1.29572936e-05, 2.20002835e-06, 2.48284186e-06,
        5.05821436e-06, 1.68418719e-08, 1.25218583e-06]], dtype=float32)

In [42]:
import numpy as np 
a = np.argmax(y_pre)
ClassName[a]

'Speed limit (30km/h)'

In [44]:
import matplotlib.pyplot as plt
plt.imshow(xtest['features'][1])

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [48]:
from tensorflow.keras.models import load_model
models = load_model('./traffic.h5')